## Side Notes for Lesson 3

### 1. Simple word embeddings with nltk and gensim

Following: https://github.com/nltk/nltk/blob/develop/nltk/test/gensim.doctest

In [ ]:
import nltk
from nltk.corpus import brown
from nltk.data import find

import gensim

import numpy as np

Define a couple of helper functions for cosine similarities: one deriving similarity between two words in the context of a model, the other for two vectors directly:

In [ ]:
def cossim_words(vec_model, a, b):
    """
    arguments: word a, word b
    return: cosine similarity between assoociated model vectors with a and b
    """
    
    vec_a = vec_model[a]
    vec_b = vec_model[b]
    
    return np.dot(vec_a, vec_b)/np.sqrt(np.dot(vec_a, vec_a))/np.sqrt(np.dot(vec_b, vec_b))

def cossim_vecs(vec_a, vec_b):
    """
    arguments: word a, word b
    return: cosine similarity between associated model vectors with a and b
    """
    
    return np.dot(vec_a, vec_b)/np.sqrt(np.dot(vec_a, vec_a))/np.sqrt(np.dot(vec_b, vec_b))

Download NLTK's sample word2vec embeddings:

In [ ]:
nltk.download('word2vec_sample')

In [ ]:
word2vec_sample = str(find('models/word2vec_sample/pruned.word2vec.txt'))

Load the embeddings into a gensim model:

In [ ]:
model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_sample, binary=False)

What is the size of the vocabulary? [Use model.vocab...]

In [ ]:
len(model.vocab)

Ok, 43981 words in vocab.

What is the **embedding**? [model['word']...]

In [ ]:
model['great']

Dimension as expected?

In [ ]:
model['great'].shape

Let's play with cosine similarities:

In [ ]:
cossim_words(model, 'nice', 'great')

In [ ]:
cossim_words(model, 'nice', 'bad')

Cool, as expected.

Now... word vectors are supposed to capture meaningful linguistic relationships. So let's try to 're-construct' the embedding vector for the word 'son' via

model['son']  $\sim$ model['boy'] - model['girl'] + model['daughter']

In [ ]:
model_son = model['boy'] - model['girl'] + model['daughter']

How close is this constructed vector to the actual embedding bector for 'boy'?

In [ ]:
cossim_vecs(model['son'], model_son)

Close! And it is much closer to the embedding of 'boy' than other words in the family (in a double-sense): 

In [ ]:
cossim_words(model, 'son', 'brother')

In [ ]:
cossim_words(model, 'son', 'daughter')

So the approximate relationship model['son']  $\sim$ model['boy'] - model['girl'] + model['daughter'] seems valid.

### 2. Simple BOW Classification using Word Embeddings in Keras

This section roughly implements the model on slides 41 in a toy setting.

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Input, Dense, Lambda
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K

Ok, now we know the number of words that have an embedding. Let's build the embedding matrix from the model:

In [ ]:
EMBEDDING_DIM = len(model['university'])      # we know... it's 300

# initialize embedding matrix and word-to-id map:
embedding_matrix = np.zeros((len(model.vocab.keys()) + 1, EMBEDDING_DIM))       
vocab_dict = {}

# build the embedding matrix and the word-to-id map:
for i, word in enumerate(model.vocab.keys()):
    embedding_vector = model[word]
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        vocab_dict[word] = i



What's the shape?

In [ ]:
embedding_matrix.shape

Correct? Looks right.

Let's build the embedding layer:

In [ ]:
MAX_SEQUENCE_LENGTH = 5  # Keras' embedding layer expects a specific input length. Padding is often needed here.

embedding_layer = Embedding(embedding_matrix.shape[0],
                            embedding_matrix.shape[1],
                            embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [ ]:
try:
    del tf_model
except:
    pass

Note the 'trainable=False' flag...

Now let's build the model, again as a **Sequential Model**: 

In [ ]:
tf_model = tf.keras.Sequential()

tf_model.add(embedding_layer)                                        # embedding layer
tf_model.add(tf.keras.layers.Lambda(lambda x: K.mean(x, axis=1)))    # average of embedding vectors
tf_model.add(Dense(100, activation='relu'))                          # hidden layer
tf_model.add(Dense(1, activation='sigmoid'))                         # classification layer

**Q: What are the dimensions of the layers?**

Next: we build the model, defining input and output:

Let's see whether our dimension discussion was correct. Print a model summary:

In [ ]:
tf_model.summary()

Like last week... let's compile the model. I.e, define optimizer, loss function, etc.

In [ ]:
tf_model.compile(optimizer='adam', loss='BinaryCrossentropy')

Almost there... let's create some fake training and test data.

In [ ]:
train_sentences = ['this is really absolutely great', 'this is really absolutely terrible']
train_labels = [[1], [0]]

test_sentences = ['never seen anything this stupid', 'never seen anything this fantastic']
test_labels = [[0], [1]]

... and then do some formatting gymnastics:

In [ ]:
def sents_to_ids(sentences):
    """
    converting a list of strings to a list of lists of word ids
    """
    text_ids = []
    for sentence in sentences:
        example = []
        for word in sentence.split(' '):
            example.append(vocab_dict[word])
        text_ids.append(example)

    return  text_ids   


train_input = np.array(sents_to_ids(train_sentences))
train_labels = np.array(train_labels)

test_input = np.array(sents_to_ids(test_sentences))
test_labels = np.array(test_labels)

So the model input are word ids in the vocab:

In [ ]:
train_input

Next: let's get the start predictions. Should be random-ish. Are they?

In [ ]:
print(tf_model.predict(train_input))
print(tf_model.predict(test_input))

Yup, looks quite random.

Finally... let's train!

In [ ]:
tf_model.fit(train_input, train_labels, validation_data=(test_input, test_labels), epochs=1)
tf_model.fit(train_input, train_labels, validation_data=(test_input, test_labels), epochs=150, verbose=0)
tf_model.fit(train_input, train_labels, validation_data=(test_input, test_labels), epochs=1)

Look's good!

What are train & test predictions now?

In [ ]:
tf_model.predict(test_input)

Yey! But we obviously cheated here with the choice of sentences. Nevertheless, the idea should be clear.

**Questions for the class for joint live in-class exercises**:

1) Can you relate the value for the validation loss to the prediction for the test set 

2) What do you think happens if you change the 'trainable' flag in the embedding layer from 'False' to 'True'?

3) Let's look into the model and inspect some weights. (Use tf_model.layers. We can get weights of individual layers through  tf_model.layers[<layer_num>].weights):
   - Related to Q2, depending on the 'trainable' flag, did the embedding matrix change?
   
   
LET'S TRY IT!!